In [1]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.cluster import MeanShift, estimate_bandwidth, get_bin_seeds
from scipy.linalg    import hankel

from tensorflow.contrib import rnn

from util.ZigZag import ZigZag, ZigZag0
from util.CalcReturns import CalcReturns

from util.BuildData import TrainingSet_NN_Prices, TrainingSet_NN_Logret
from util.BuildData import TrainingSet_ML_Prices, TrainingSet_ML_Logret

In [2]:
class Grid:
    def __init__(self, dx = 8.448750494999999e-06, dy = 8.44596043e-06, bound = 0.002):
        
        self.dx = dx
        self.dy = dy
        
        self.xmin = -math.fabs(bound)
        self.xmax =  math.fabs(bound)
        
        self.ymin = -math.fabs(bound)
        self.ymax =  math.fabs(bound)
        
        self.__recalculate()        
        pass
    
    def fit(self, X):        
        labels   = []
        clusters = []
        
        for i in range(len(X)):            
            _, _, index, cell = self.cell(X[i])
            
            labels.append(index)
            clusters.append(cell)    
        
        labels   = np.reshape(labels,   (len(labels),  1))
        clusters = np.reshape(clusters, (len(clusters),2))
        
        return labels, clusters
    
    def fit_cells(self, X):
        cells = []
        
        for i in range(len(X)):            
            i, j, _, _ = self.cell(X[i])
            cells.append([i, j])
        
        cells = np.reshape(cells, (len(cells),2))        
        return cells
    
    def cell(self, v):
        dims = math.sqrt(grid.shape()[0])
        
        dy   = self.dy
        dy_2 = dy/2
        
        dx   = self.dx
        dx_2 = dx/2
        
        v0 = v[0]
        v1 = v[1]
        
        sv0 = 0
        if v0 != 0.0:
            sv0 = int(v0 / math.fabs(v0))
        
        sv1 = 0
        if v1 != 0.0:
            sv1 = int(v1 / math.fabs(v1))
        
        i0 = int((v0 + sv0 * dy_2) / dy) 
        j0 = int((v1 + sv1 * dx_2) / dx) 
        
        i = i0 + int(dims/2)
        j = j0 + int(dims/2)

        index = int(i*dims + j)
        cell  = self.grid[index]
        
        return i0, j0, index, cell
    
    def __recalculate(self):
        x = 0
        grid_x = []
        grid_x.append(x)

        while x < self.xmax:        
            x = x + self.dx
            grid_x.append(x)
            grid_x.append(-x)
    
        y = 0
        grid_y = []
        grid_y.append(y)

        while y < self.ymax:
            y = y + self.dy
            grid_y.append(y)
            grid_y.append(-y)
    
        grid_x.sort()
        grid_y.sort()

        grid = []

        for i in range(0,len(grid_y)):
            for j in range(0,len(grid_x)):
                grid.append([grid_y[i], grid_x[j]])

        self.grid = np.array(grid)
        pass
        
    def plot(self, show = False):
        G = self.grid.T        
        plt.scatter(G[0], G[1], color='red')
        
        if show:
            plt.show()
        
        pass
    
    def shape(self):
        return self.grid.shape

def metric(x):
    return max(math.fabs(x[0]), math.fabs(x[1]))

def build_set(S, lag = 2):
    return hankel(S[0 : lag], S[lag-1 :]).T

In [3]:
def recurrent_neural_network_model(input):

    initial_w_real = tf.random_normal([num_hidden, n_classes])
    initial_w_imag = tf.random_normal([num_hidden, n_classes])
    
    initial_w = tf.complex(initial_w_real, initial_w_imag)
    
    initial_b_real = tf.random_normal([n_classes])
    initial_b_imag = tf.random_normal([n_classes])
    
    initial_b = tf.complex(initial_b_real, initial_b_imag)
    
    layer = {'w' : tf.Variable(initial_w, dtype=tf.complex64),
             'b' : tf.Variable(initial_b, dtype=tf.complex64)}
    
    input = tf.unstack(input, timesteps, 1)
    
    lstm_cell = rnn.LSTMCell(num_hidden)
    #state = lstm_cell.zero_state([timesteps], dtype=tf.complex64)
    
    #outs, state = rnn.static_rnn(lstm_cell, input, initial_state=state, dtype=tf.complex64)
    outs, state = rnn.static_rnn(lstm_cell, input, dtype=tf.complex64)
    
    output = tf.add(tf.matmul(outs[-1], layer['w']), layer['b'], name='nn')    
    return output

def train_and_save_nn(x_data,y_data, filename):
    x_train = tf.placeholder(shape=[None, timesteps, num_input], name='x', dtype=tf.complex64)
    y_train = tf.placeholder(shape=[None, n_classes], name='y', dtype=tf.complex64)
    
    nn = recurrent_neural_network_model(x_train)            
    cost = tf.norm(tf.subtract(nn, y_train), name='cost')
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)   
    
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        
        session.run(tf.initialize_all_variables())
        
        epoch_loss = 0
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int(len(X_train)/batch_size)):
                
                batch_x = x_data[i*batch_size : (i+1)*batch_size]
                batch_y = y_data[i*batch_size : (i+1)*batch_size]
               
                batch_x = batch_x.reshape((batch_size, timesteps, num_input))
                
                _, c = session.run([optimizer, cost], feed_dict = {x_train: batch_x, y_train: batch_y})
                epoch_loss += c
                
            if epoch % 100 == 0:
                print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
               
        saver.save(session, filename)
        print('Last epoch loss: ', epoch_loss)
        
    return nn#, correct, accuracy

def test_nn(x_test, y_test, filename):
    with tf.Session() as session:        
        saver = tf.train.import_meta_graph(filename)
        saver.restore(session, tf.train.latest_checkpoint('data/'))
        
        graph    = tf.get_default_graph()        
                
        x  = graph.get_tensor_by_name('x:0')
        y  = graph.get_tensor_by_name('y:0')
        
        nn = graph.get_tensor_by_name('nn:0')        

        inputs  = x_test        
        inputs  = np.reshape(inputs, (testsize, timesteps, num_input))
        
        outputs = session.run([nn], feed_dict = {x: inputs})
        predicted = outputs[0]        
        
        #x0 = y_test[0]
        #x0 = np.reshape(x0, (1, n_classes))
        
        #y0 = predicted[0]
        #y0 = np.reshape(y0, (1, n_classes))
        
        #x_ = tf.placeholder('float', [1, n_classes])
        #y_ = tf.placeholder('float', [1, n_classes])        
        
        #print x0
        #print y0
        
        #cost = tf.norm(tf.subtract(x_, y_))
        #output = session.run([cost], feed_dict = {x_: x0, y_: y0})
        #print output
        
        # Visualising the results
        
        print y_test
        print predicted
        
        plt.plot(y_test, 'b*',    label = 'Actual')
        plt.plot(predicted, 'r+', label = 'Predicted')

        plt.title('Prediction')        
        plt.legend()
        plt.show()

In [4]:
%matplotlib

N = 8*1440 + 1

source = pd.read_csv('EURUSD1.csv', header=0) # source CSV
prices = np.array(source.Close)[0:N+1] # close prices
r = CalcReturns(prices)  


Using matplotlib backend: TkAgg


In [5]:
X = build_set(r)

grid = Grid(bound = 0.004)
#grid.plot()

#plot_phase_space0(r, N-1)
#plt.show()

cells = grid.fit_cells(X)

Real = cells.T[0]
Imag = cells.T[1]

C = Real + 1j*Imag



#C = np.reshape(cells.T, (len(cells), 2,1))

#print C

#plt.plot(labels)
#plt.show()

In [6]:
timesteps  = 1
num_input  = 1
n_classes  = 1
step       = num_input * timesteps

lag        = 1
testsize   = 16
trainsize  = 2*1440 + 16 #N - testsize

hm_epochs  = 4000
num_hidden = 1440 / 2
batch_size = 512

In [7]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()

# Importing the training set
data_set = C[:trainsize + lag + testsize + lag + 1]
#data_set = sc.fit_transform(data_set)

testing_set  = data_set[-testsize-lag:]
training_set = data_set[:trainsize+lag]

# Getting the inputs and the ouputs
X_train = training_set[0:trainsize]

Y_train = training_set[lag:trainsize+lag]
Y_train = np.reshape(Y_train, (len(Y_train), n_classes))

X_test = testing_set[0:testsize]

Y_test = testing_set[lag:testsize+lag]
Y_test = np.reshape(Y_test, (len(Y_test), n_classes))

# Reshaping
X_train = np.reshape(X_train, (trainsize, timesteps, num_input))

In [8]:
print X_train
print X_train.shape

train_and_save_nn(X_train, Y_train, 'data/fxrn_0.ckpt')

[[[  0.-33.j]]

 [[-33.+46.j]]

 [[ 46.+12.j]]

 ..., 
 [[  3. +7.j]]

 [[  7. -3.j]]

 [[ -3. -2.j]]]
(2896, 1, 1)
[<tf.Tensor 'unstack:0' shape=(?, 1) dtype=complex64>]


ValueError: An initializer for variable rnn/lstm_cell/kernel of <dtype: 'complex64'> is required